In [1]:
import os
os.chdir('/Users/yinancheng/Documents/Project/TrustworthyAI/Advertisement/')

In [35]:
import pandas as pd 
path = 'Data' # Path of the dataset folder
task_id = 31941 # Pick a task

df_feeds = pd.read_csv(path + '/train/train_data_feeds.csv')
df_ads = pd.read_csv(path + '/train/train_data_ads.csv')
print("data is loaded...")

data is loaded...


In [36]:
user_ads = df_ads['user_id'].unique()
df_feeds_pot = df_feeds[df_feeds['u_userId'].isin(user_ads)].sort_values(by='e_et')

In [37]:
print(df_feeds_pot.shape)
print(df_feeds.shape)
print(df_ads.shape)

(2475843, 28)
(3227732, 28)
(7675517, 35)


In [60]:
df_ads_task = df_ads[df_ads['task_id'] == task_id].sort_values(by = 'pt_d')

Total Sample size is 77470, Positive Sample size is 2492, Negative Sample size is 74978, label rate is 0.033236416015364506


In [61]:
df_merged = pd.merge_asof(df_ads_task, df_feeds_pot, left_on='pt_d', right_on='e_et', left_by = 'user_id', right_by = 'u_userId', direction='backward')
df_merged = df_merged.sample(frac=1).reset_index(drop=True)

In [62]:
import numpy as np
columns_to_drop = [column for column in df_merged.columns if df_merged[column].nunique() > 1000 or df_merged[column].nunique() == 1]

df_merged = df_merged.drop(columns=columns_to_drop)

df_merged = df_merged.select_dtypes(include=[np.number])

df_merged = df_merged.dropna()
df_merged = df_merged.astype(int)

print(df_merged.nunique())
print(df_merged.shape)

label_x                 2
age                     8
gender                  3
residence              34
city                  332
city_rank               4
series_dev             23
series_group            7
emui_dev               24
device_name           217
device_size           552
creat_type_cd           2
slot_id                49
u_refreshTimes_x       10
u_feedLifeCycle_x       8
u_phonePrice            7
u_browserLifeCycle      8
u_browserMode           7
u_feedLifeCycle_y       8
u_refreshTimes_y       10
i_regionEntity        252
i_cat                 193
i_dislikeTimes         10
i_upTimes              10
i_dtype                 5
e_ch                   19
e_m                   217
e_po                   23
e_rn                   56
e_section               2
label_y                 2
cillabel                2
pro                    11
dtype: int64
(56342, 33)


In [63]:
from Dataset_Utility.utility_functions import calculate_label_rate2, get_train_holdout_validate

calculate_label_rate2(df_merged, 'label_x')
n = df_merged.shape[0]#sample size we want
df_train, df_holdout, df_val = get_train_holdout_validate(df_merged, 'label_x', n)

calculate_label_rate2(df_train,'label_x')
calculate_label_rate2(df_holdout,'label_x')
calculate_label_rate2(df_val,'label_x')

task_path = f'{path}/potential_{task_id}'
if not os.path.exists(task_path):
    os.makedirs(task_path)

df_train.to_csv(f'{task_path}/df_train.csv', index=False)
df_holdout.to_csv(f'{task_path}/df_holdout.csv', index=False)
df_val.to_csv(f'{task_path}/df_val.csv', index=False)

Total Sample size is 56342, Positive Sample size is 1272, Negative Sample size is 55070, label rate is 0.023097875431269295
Total Sample size is 22536, Positive Sample size is 508, Negative Sample size is 22028, label rate is 0.023061558017069186
Total Sample size is 22536, Positive Sample size is 508, Negative Sample size is 22028, label rate is 0.023061558017069186
Total Sample size is 11268, Positive Sample size is 254, Negative Sample size is 11014, label rate is 0.023061558017069186
